### Create a Game UI with Gradio

In [7]:
import gradio as gr  #UI tools
import os
demo = None #added to allow restart

def start_game(main_loop, share=False):
    # added code to support restart
    global demo
    # If demo is already running, close it first
    if demo is not None:
        demo.close()

    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=250, placeholder="Type 'start' to begin game"),
        textbox=gr.Textbox(placeholder="Enter The Next Prompt?", container=False, scale=7),
        title="AI RPG",
        # description="Ask Yes Man any question",
        theme="soft",
        examples=["Take an action","Look around", "Continue the story", "Pick up an object"],
        cache_examples=False,
        retry_btn="Retry",
        undo_btn="Undo",
        clear_btn="Clear",
                           )
    demo.launch(share=share, server_name="0.0.0.0")

def test_main_loop(message, history):
    return 'Entered Action: ' + message

start_game(test_main_loop)

ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7860): address already in use
ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7861): address already in use


Running on local URL:  http://0.0.0.0:7862

To create a public link, set `share=True` in `launch()`.


### Generating an Initial Start

In [5]:
from helper import load_world, save_world
from together import Together
from helper import get_together_api_key, load_env

client = Together(api_key=get_together_api_key())

world = load_world('../AI-Game-Creator/Kyropeia.json') 
kingdom = world['kingdoms']['Eldrida']
town = kingdom['towns']["Luminaria"]
character = town['npcs']['Elwyn Moonwhisper']

In [6]:
system_prompt = """You are an AI Game master. Your job is to create a 
start to an adventure based on the world, kingdom, town and character 
a player is playing as. 
Instructions:
You must only use 2-4 sentences \
Write in second person. For example: "You are Jack" \
Write in present tense. For example "You stand at..." \
First describe the character and their backstory. \
Then describes where they start and what they see around them."""
world_info = f"""
World: {world}
Kingdom: {kingdom}
Town: {town}
Your Character: {character}
"""

In [8]:
model_output = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    temperature=1.0,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info + '\nYour Start:'}
    ],
)


In [11]:
start = model_output.choices[0].message.content
print(start)
world['start'] = start
#save_world(world, '../shared_data/Kyropeia.json')  # preserve video version
#save_world(world, '../shared_data/YourWorld_L1.json')

You stand atop a winding staircase, gazing out upon the radiant city of Luminaria, its buildings and streets shimmering with an ethereal glow. As Elwyn Moonwhisper, a 30-year-old Colossi whisperer, you've grown accustomed to the whispers of the ancient creatures, sensing their emotions and desires. Your gentle nature belies a deep sense of purpose, driven by a desire to protect the Colossi from those who would exploit their power, and to find a long-lost sibling who was taken by the very creatures you've sworn to defend. You've received a cryptic message hinting at a hidden gathering of Colossi sympathizers within the city's labyrinthine libraries, and you sense that this meeting may hold the key to unlocking secrets about your sibling's disappearance.


### Creating the Main Action Loop

In [12]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""
    
    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [13]:
game_state = {
    "world": world['description'],
    "kingdom": kingdom['description'],
    "town": town['description'],
    "character": character['description'],
    "start": start,
}

def main_loop(message, history):
    return run_action(message, history, game_state)

### Launch the Game!

In [14]:
start_game(main_loop, True)

Closing server running on port: 7862


ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7860): address already in use
ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7861): address already in use


Running on local URL:  http://0.0.0.0:7862
Running on public URL: https://1473bab27aa2242b9f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
